In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from adamp import AdamP
from transformers.optimization import get_cosine_schedule_with_warmup

import random

##GPU 사용 시
device = torch.device("cuda:0")

## 시드 고정

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()


## 데이터 로딩

In [4]:
# 학습용 데이터셋 불러오기
import pandas as pd
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

print(train.shape)
print(test.shape)

(45654, 3)
(9131, 2)


In [5]:
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [6]:
# index 컬럼 제거
train = train.iloc[:,1:]
test = test.iloc[:,1:]


## 텍스트 데이터 전처리

In [7]:
from konlpy.tag import Okt

In [8]:
# 형태소 분석기(Okt) 불러오기 
okt=Okt() 

In [9]:
'''# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            clean.append(word[0])
    
    
    return " ".join(clean) 

train['title'] = train['title'].apply(lambda x : func(x))
test['title'] = test['title'].apply(lambda x : func(x))'''

'# 조사, 어미, 구두점 제거\ndef func(text):\n    clean = []\n    for word in okt.pos(text, stem=True): #어간 추출\n        if word[1] not in [\'Josa\', \'Eomi\', \'Punctuation\']: #조사, 어미, 구두점 제외 \n            clean.append(word[0])\n    \n    \n    return " ".join(clean) \n\ntrain[\'title\'] = train[\'title\'].apply(lambda x : func(x))\ntest[\'title\'] = test[\'title\'].apply(lambda x : func(x))'

In [12]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("valid shape is:", len(valid))

train shape is: 36523
valid shape is: 9131


## Custom dataset

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

#         self.sentences = [transform([i[sent_idx]]) for i in dataset.to_numpy()]
        self.token_ids = []
        self.valid_length =[]
        self.segment_ids = []
        for i in dataset.to_numpy():
            out = transform([i[sent_idx]])
            self.token_ids.append(out[0])
            self.valid_length.append(out[1])
            self.segment_ids.append(out[2])
        self.labels = [np.int32(i[label_idx]) for i in dataset.to_numpy()]
        

    def __getitem__(self, i):
        token_ids = self.token_ids[i]
        valid_length = self.valid_length[i]
        segment_ids = self.segment_ids[i]
        return {
            'token_ids' : torch.tensor(token_ids, dtype=torch.long),
            'valid_length' : torch.tensor(valid_length, dtype=torch.long),
            'segment_ids' : torch.tensor(segment_ids, dtype=torch.long),
            'label' : torch.tensor(self.labels[i], dtype=torch.long)
        }

    def __len__(self):
        return (len(self.labels))


## 파라미터 셋팅

In [24]:
# Setting parameters
max_len = 48 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

## 모델링

In [25]:
bertmodel, vocab = get_pytorch_kobert_model()

# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        bert_output = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = bert_output['pooler_output']
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      
model = BERTClassifier(bertmodel, dr_rate=0.7).cuda()

In [27]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(valid, 0, 1, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)



## 학습에 필요한 파라미터 정의 

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamP(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능


#scheduler
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
scaler = torch.cuda.amp.GradScaler()

In [29]:

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  

In [30]:
epoch_num = 0


## 모델 학습

In [31]:
valid_acc_max = 0
# 모델 학습 시작
model.train()

for e in range(num_epochs):
    epoch_num+=1
    train_acc = 0.0
    test_acc = 0.0
    tqdm_dataset = tqdm(enumerate(train_dataloader))
    for batch_id, items in tqdm_dataset:
        
        optimizer.zero_grad()
        token_ids = items['token_ids'].cuda()
        segment_ids = items['segment_ids'].cuda()
        valid_length= items['valid_length']
        label = items['label'].cuda()
        with torch.cuda.amp.autocast():
            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)
        #loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        #optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    model.eval() # 평가 모드로 변경
    tqdm_dataset = tqdm(enumerate(test_dataloader))
    for batch_id, items in tqdm_dataset:
        token_ids = items['token_ids'].cuda()
        segment_ids = items['segment_ids'].cuda()
        valid_length= items['valid_length']
        label = items['label'].cuda()
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

0it [00:00, ?it/s]

epoch 1 batch id 1 loss 2.0181121826171875 train acc 0.1484375
epoch 1 batch id 201 loss 0.29367995262145996 train acc 0.636738184079602
epoch 1 train acc 0.705152768742885


0it [00:00, ?it/s]

epoch 1 test acc 0.8803244105297158


0it [00:00, ?it/s]

...

epoch 4 test acc 0.8884599079457365


0it [00:00, ?it/s]

epoch 5 batch id 1 loss 0.046438053250312805 train acc 0.9921875
epoch 5 batch id 201 loss 0.03527696430683136 train acc 0.9808379975124378
epoch 5 train acc 0.9818072552447552


0it [00:00, ?it/s]

epoch 5 test acc 0.8880258801679587



## test 데이터 추론

In [32]:
import time
import datetime
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [33]:
import time
import datetime
# 테스트 데이터 예측

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

pred = []
for step in range(len(test)):
    if step % 500 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step,len(test), elapsed))
    test_sentence = test.title[step]
    test_label = 0


    unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['title', 'topic_idx']])
    #unseen_values = unseen_test.values
    test_set = BERTDataset(unseen_test, 0, 1, tok, max_len, True, False)
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

    for batch_id, items in enumerate(test_input):
        token_ids = items['token_ids'].cuda()
        segment_ids = items['segment_ids'].cuda()
        valid_length= items['valid_length']
        #label = items['label'].to(device)
        out = model(token_ids, valid_length, segment_ids)
        pred.append(int(torch.argmax(out).cpu().numpy()))

  Batch   500  of  9,131.    Elapsed: 0:00:05.
  Batch 1,000  of  9,131.    Elapsed: 0:00:11.
  Batch 1,500  of  9,131.    Elapsed: 0:00:16.
  Batch 2,000  of  9,131.    Elapsed: 0:00:22.
  Batch 2,500  of  9,131.    Elapsed: 0:00:27.
  Batch 3,000  of  9,131.    Elapsed: 0:00:33.
  Batch 3,500  of  9,131.    Elapsed: 0:00:38.
  Batch 4,000  of  9,131.    Elapsed: 0:00:44.
  Batch 4,500  of  9,131.    Elapsed: 0:00:49.
  Batch 5,000  of  9,131.    Elapsed: 0:00:54.
  Batch 5,500  of  9,131.    Elapsed: 0:01:00.
  Batch 6,000  of  9,131.    Elapsed: 0:01:05.
  Batch 6,500  of  9,131.    Elapsed: 0:01:11.
  Batch 7,000  of  9,131.    Elapsed: 0:01:16.
  Batch 7,500  of  9,131.    Elapsed: 0:01:22.
  Batch 8,000  of  9,131.    Elapsed: 0:01:27.
  Batch 8,500  of  9,131.    Elapsed: 0:01:32.
  Batch 9,000  of  9,131.    Elapsed: 0:01:38.



## 제출 파일 생성

In [34]:
sub = pd.read_csv('sample_submission.csv')
sub['topic_idx'] = pred
sub.to_csv('kobert_epcoh5.csv',index= False)

In [35]:
sub.head()

,index,topic_idx
0,45654,3
1,45655,3
2,45656,2
3,45657,2
4,45658,3


In [36]:
sol =  pd.read_csv('solution_sample.csv')
#sol2 = pd.read_csv('solution.csv')

sub2 = pred[0:len(sol)]

from sklearn.metrics import accuracy_score
print(accuracy_score(sol.topic_idx, sub2))
print(epoch_num,"번째 완료")

0.8552026286966046
5 번째 완료
